In [1]:
import numpy as np
import sys
import os

# Absolute path to the folder that contains "quant_rotor"
sys.path.append("/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems")

#working with kietic and potential operators

from quant_rotor.models.dense.support_ham import write_matrix_elements, basis_m_to_p_matrix_conversion

from quant_rotor.models.sparse.support_ham import build_V_in_p

#ED hamiltonian implementation.
from quant_rotor.core.dense.hamiltonian import hamiltonian_dense
from quant_rotor.core.dense.hamiltonian_big import hamiltonian_general_dense

# Iterative residual calculation
from quant_rotor.core.dense.t_amplitudes_guess import amplitute_energy, t_1_amplitude_guess_ground_state, t_2_amplitude_guess_ground_state
from quant_rotor.core.dense.t_amplitudes_periodic import t_periodic
from quant_rotor.core.dense.t_amplitudes_non_periodic import t_non_periodic

from quant_rotor.core.dense.t_amplitudes_periodic_fast import t_periodic # for bigger systems (much harder to modify)

#Time dependant methods
from quant_rotor.core.dense.de_solve_one_thermal import integration_scheme

from quant_rotor.core.dense.de_solve_one_thermal_dense import integration_scheme as integration_scheme_fast # for bigger systems (much harder to modify)

# Producing hamiltonian operators.

## Producing Kinetic and Potential operators.

In [26]:
state = 3
g = 0.1

Example of the call to produce kinetic and potential matricies.

In [27]:
# Recomended to produce small matrix which can be printed and see how it changes. For example 3 state system

Kinetic, Potential = write_matrix_elements((state-1)//2) # takes input of the number of unique states see doc-string and annotation for more information

K, V = Kinetic, Potential

V_tensor = V.reshape(state, state, state, state)  # Adjust if needed

h_full = basis_m_to_p_matrix_conversion(K, state)
v_full = basis_m_to_p_matrix_conversion(V_tensor, state)

v_full = v_full * g

In [28]:
h_full

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [29]:
v_full.reshape(state**2, state**2)

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.075, -0.025,  0.   , -0.025,  0.075],
       [ 0.   ,  0.   ,  0.   , -0.025,  0.   ,  0.   ,  0.075,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.075,  0.   ,  0.   , -0.025,  0.   ,  0.   ],
       [ 0.   , -0.025,  0.075,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.075,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.025,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.075, -0.025,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.025,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.075,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [30]:
Kinetic_sparse, Potential_sparse = build_V_in_p(state) # the fast version give a result in sparse which needs to ve converted to dense matricies

# Example of how to convert to dense.

Kinetic_dense = Kinetic_sparse.toarray()
Potential_dense = Potential_sparse.toarray()

In [31]:
Kinetic_dense

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [32]:
Potential_dense

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.075, -0.025,  0.   , -0.025,  0.075],
       [ 0.   ,  0.   ,  0.   , -0.025,  0.   ,  0.   ,  0.075,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.075,  0.   ,  0.   , -0.025,  0.   ,  0.   ],
       [ 0.   , -0.025,  0.075,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.075,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.025,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.075, -0.025,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.025,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.075,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

Check the output.

In [33]:
print(np.array_equal(h_full, Kinetic_dense))
print(np.allclose(v_full.reshape(state**2, state**2), Potential_dense), 1e-30)

True
True 1e-30


Produce several matricies for tests.

In [36]:
for state in range(3, 15, 2):
    
    Kinetic_sparse, Potential_sparse = build_V_in_p(state) # the fast version give a result in sparse which needs to ve converted to dense matricies

    # Example of how to convert to dense.

    Kinetic_dense = Kinetic_sparse.toarray()
    Potential_dense = Potential_sparse.toarray()

    np.savez(f"example_data.kinetic_potential_matricies.state_{state}.npz", K=Kinetic_dense, V=Potential_dense)

In [38]:
for state in range(3, 15, 2):

    Kinetic_sparse, Potential_sparse = build_V_in_p(state) # the fast version give a result in sparse which needs to ve converted to dense matricies

    # Example of how to convert to dense.

    Kinetic_dense = Kinetic_sparse.toarray()
    Potential_dense = Potential_sparse.toarray()

    data = np.load(f"example_data.kinetic_potential_matricies.state_{state}.npz")

    print(np.array_equal(Kinetic_dense, data["K"]))
    print(np.array_equal(Potential_dense, data["V"]))

True
True
True
True
True
True
True
True
True
True
True
True


## Producing hamiltonians.

In [12]:
site = 3
state = 11
g = 0.1

Exact Diognalisation hamiltonian.

In [13]:
Hamiltonian, K, V = hamiltonian_dense(state, site, g)

H = Hamiltonian

eig_val_ED, eig_vec_ED = np.linalg.eigh(H) # produces the solutions to the hamiltonian with energys and associated wavefunctions.

Natural Orbitals hamiltonian.

In [14]:
H, K, V = hamiltonian_general_dense(state, site, g)

eig_val_NO, eig_vec_NO = np.linalg.eigh(H) # produces the solutions to the hamiltonian with energys and associated wavefunctions.

A way to check the NO with ED we can check the first #states biggest energyes and eigenstates.

In [15]:
index_ED = np.argsort(-eig_val_ED)
index_NO = np.argsort(-eig_val_NO)

# Makes a change of basis matrix.
np.array_equal(eig_vec_ED[:, index_ED[:state]], eig_vec_NO[:, index_NO[:state]])

False

In [18]:
eig_val_ED[index_ED[:state]]

array([75.003729325757, 75.003729325757, 75.001513456434, 75.001513456434, 75.001513456434, 75.001513456434, 75.001513456434, 75.001513456434, 66.189688856973, 66.189688856973, 66.189688856973])

In [17]:
eig_val_NO[index_NO[:state]]

array([75.003729325757, 75.003729325756, 75.001513456434, 75.001513456434, 75.001513456434, 75.001513456434, 75.001513456434, 75.001513456434, 66.189688856973, 66.189688856973, 66.189688856973])

In [19]:
eig_vec_ED[:, index_ED[:state]]

array([[-0.            +0.j, -0.            +0.j, -0.            +0.j, ...,  0.            +0.j, -0.            +0.j,  0.            +0.j],
       [-0.            +0.j, -0.            +0.j,  0.            +0.j, ..., -0.            +0.j,  0.            +0.j,  0.            +0.j],
       [ 0.            +0.j, -0.            +0.j,  0.            +0.j, ...,  0.            +0.j, -0.            +0.j, -0.            +0.j],
       ...,
       [-0.            +0.j, -0.            +0.j, -0.            +0.j, ...,  0.            +0.j,  0.            +0.j,  0.            +0.j],
       [-0.000000000001+0.j, -0.000000000007+0.j, -0.91046154519 +0.j, ..., -0.            +0.j, -0.            +0.j, -0.            +0.j],
       [ 0.001492357015+0.j, -0.999895866921+0.j, -0.            +0.j, ..., -0.            +0.j,  0.            +0.j,  0.            +0.j]], shape=(1331, 11))

In [20]:
eig_vec_NO[:, index_NO[:state]]

array([[ 0.            +0.j, -0.            +0.j, -0.            +0.j, ...,  0.            +0.j, -0.            +0.j, -0.            +0.j],
       [-0.            +0.j, -0.            +0.j, -0.            +0.j, ..., -0.            +0.j,  0.            +0.j,  0.            +0.j],
       [ 0.            +0.j, -0.            +0.j,  0.            +0.j, ..., -0.            +0.j,  0.            +0.j,  0.            +0.j],
       ...,
       [ 0.            +0.j, -0.            +0.j,  0.            +0.j, ..., -0.32988280112 +0.j, -0.000000250166+0.j, -0.054590162904+0.j],
       [-0.272429254153+0.j, -0.419202412339+0.j,  0.076034323134+0.j, ..., -0.000000000001+0.j,  0.            +0.j, -0.000000000001+0.j],
       [ 0.419202410484+0.j, -0.272429252892+0.j,  0.267566110008+0.j, ...,  0.            +0.j,  0.000000000002+0.j,  0.            +0.j]], shape=(1331, 11))